# MTG Card Scoring Algorithm - Critical Fixes Validation

**Date:** January 23, 2026  
**Purpose:** Validate fixes for:
1. Color identity enforcement (strict Commander format rules)
2. Mechanic synergy calculation (debug always-100 issue)
3. Card deduplication (remove duplicate cards from results)

## Critical Validation Checklist

- [ ] Color identity enforcement working (illegal cards get multiplier = 0.0)
- [ ] No off-color cards in top 100 recommendations
- [ ] Synergy scores have meaningful variance (not all 100)
- [ ] No duplicate card names in results
- [ ] Different commanders get different recommendations

## Section 1: Load Data and Initialize Scorer

Load the cards dataset and initialize CardScorer with color identity lookup from CosmosDB.

In [1]:
import pandas as pd
import numpy as np
import sys
import logging

# Setup logging to see diagnostic messages
logging.basicConfig(
    level=logging.DEBUG,
    format='%(levelname)s: %(message)s'
)

# Add project to path
sys.path.insert(0, '/workspaces/mtgecorec')

from core.data_engine.card_scoring import CardScorer

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [2]:
# Initialize CardScorer
print("Initializing CardScorer...")
scorer = CardScorer(data_path='/workspaces/mtgecorec/notebooks/')

print(f"✓ CardScorer initialized")
print(f"  - Total cards loaded: {len(scorer.all_cards)}")
print(f"  - Color identity lookup entries: {len(scorer.color_identity_lookup)}")
print(f"  - Mechanic weights loaded: {len(scorer.mechanic_weight_lookup)}")

# DEBUG: Show what's actually in the lookup
if len(scorer.color_identity_lookup) > 0:
    sample_items = list(scorer.color_identity_lookup.items())[:3]
    print(f"\n  Sample color identity data:")
    for card_name, colors in sample_items:
        print(f"    {card_name}: {colors}")
else:
    print(f"\n  ⚠ WARNING: Color identity lookup is EMPTY!")
    print(f"    This means CosmosDB load failed or returned no data")

INFO: Initializing CardScorer from /workspaces/mtgecorec/notebooks/
INFO: Loaded 73063 cards from master_analysis_full.csv
INFO: Loaded 50 mechanic weights
INFO: Loaded 700 archetype-mechanic mappings
INFO: Loaded 12061 combo cards


Initializing CardScorer...


INFO: Loaded mechanic co-occurrence matrix
INFO: Loading color identity from CosmosDB cards collection...
INFO: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
DEBUG: {"message": "Starting topology monitoring", "topologyId": {"$oid": "69743a584abf16cc17ce47e6"}}
DEBUG: {"message": "Topology description changed", "topologyId": {"$oid": "69743a584abf16cc17ce47e6"}, "previousDescription": "<TopologyDescription id: 69743a584abf16cc17ce47e6, topology_type: Unknown, servers: []>", "newDescription": "<TopologyDescription id: 69743a584abf16cc17ce47e6, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mtgecorec.mongo.cosmos.azure.com', 10255) server_type: Unknown, rtt: None>]>"}
DEBUG: {"message": "Starting server monitoring", "topologyId": {"$oid": "69743a584abf16cc17ce47e6"}, "serverHost": "mtgecorec.mongo.cosmos.azure.com", "serverPort": 10255}
DEBU

✓ CardScorer initialized
  - Total cards loaded: 110031
  - Color identity lookup entries: 35510
  - Mechanic weights loaded: 50

  Sample color identity data:
    Forest: {'G'}
    Fury Sliver: {'R'}
    Kor Outfitter: {'W'}


In [3]:
# Verify CosmosDB connection and color_identity field
print("\n" + "="*70)
print("VERIFYING COSMOSDB COLOR IDENTITY FIELD")
print("="*70)

try:
    from core.data_engine.cosmos_driver import get_mongo_client, get_collection
    
    client = get_mongo_client()
    import os
    db_name = os.environ.get('COSMOS_DB_NAME', 'cards')
    collection_name = 'cards'
    
    cards_collection = get_collection(client, db_name, collection_name)
    
    # Check collection size
    count = cards_collection.count_documents({})
    print(f"✓ Connection to CosmosDB successful")
    print(f"  Database: {db_name}")
    print(f"  Collection: {collection_name}")
    print(f"  Total documents: {count}")
    
    if count > 0:
        # Get a sample document and show its structure
        sample = cards_collection.find_one()
        print(f"\n  Sample document keys: {list(sample.keys())}")
        
        # Check specifically for color_identity
        if 'color_identity' in sample:
            print(f"  ✓ color_identity field FOUND: {sample.get('color_identity')}")
        else:
            print(f"  ✗ color_identity field NOT FOUND in database!")
            print(f"    Available fields: {list(sample.keys())}")
    else:
        print(f"\n  ✗ No cards found in collection!")
        
except Exception as e:
    print(f"✗ Exception during CosmosDB verification: {e}")
    import traceback
    traceback.print_exc()

print("="*70)

INFO: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
DEBUG: {"message": "Starting topology monitoring", "topologyId": {"$oid": "69743a6c4abf16cc17ce47e8"}}
DEBUG: {"message": "Topology description changed", "topologyId": {"$oid": "69743a6c4abf16cc17ce47e8"}, "previousDescription": "<TopologyDescription id: 69743a6c4abf16cc17ce47e8, topology_type: Unknown, servers: []>", "newDescription": "<TopologyDescription id: 69743a6c4abf16cc17ce47e8, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mtgecorec.mongo.cosmos.azure.com', 10255) server_type: Unknown, rtt: None>]>"}
DEBUG: {"message": "Starting server monitoring", "topologyId": {"$oid": "69743a6c4abf16cc17ce47e8"}, "serverHost": "mtgecorec.mongo.cosmos.azure.com", "serverPort": 10255}
DEBUG: {"message": "Connection pool created", "clientId": {"$oid": "69743a6c4abf16cc17ce47e8"}, "serverHost": 


VERIFYING COSMOSDB COLOR IDENTITY FIELD


DEBUG: {"message": "Server heartbeat started", "topologyId": {"$oid": "69743a6c4abf16cc17ce47e8"}, "driverConnectionId": 1, "serverHost": "mtgecorec-westus2.mongo.cosmos.azure.com", "serverPort": 10255, "awaited": false}
DEBUG: {"message": "Server heartbeat succeeded", "topologyId": {"$oid": "69743a6c4abf16cc17ce47e8"}, "driverConnectionId": 1, "serverConnectionId": 58200271, "serverHost": "mtgecorec-westus2.mongo.cosmos.azure.com", "serverPort": 10255, "awaited": false, "durationMS": 11.343863006914034, "reply": "{\"ismaster\": true, \"maxBsonObjectSize\": 16777216, \"maxMessageSizeBytes\": 48000000, \"maxWriteBatchSize\": 1000, \"localTime\": 1769224813143, \"logicalSessionTimeoutMinutes\": 30, \"maxWireVersion\": 21, \"tags\": {\"region\": \"West US 2\"}, \"hosts\": [\"mtgecorec-westus2.mongo.cosmos.azure.com:10255\"], \"setName\": \"globaldb\", \"setVersion\": 1, \"primary\": \"mtgecorec-westus2.mongo.cosmos.azure.com:10255\", \"me\": \"mtgecorec-westus2.mongo.cosmos.azure.com:1025

✓ Connection to CosmosDB successful
  Database: mtgecorec
  Collection: cards
  Total documents: 110031

  Sample document keys: ['_id', 'object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'arena_id', 'tcgplayer_id', 'cardmarket_id', 'name', 'lang', 'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'colors', 'color_identity', 'keywords', 'produced_mana', 'legalities', 'games', 'reserved', 'game_changer', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'collector_number', 'digital', 'rarity', 'card_back_id', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame', 'full_art', 'textless', 'booster', 'story_spotlight', 'prices', 'related_uris', 'purchase_uris', 'day_uploaded', 'pricing', 'last_price_update']
  ✓ color_identity f

## Section 2: Color Identity Enforcement

Test strict Commander format color identity enforcement.

In [4]:
# Test extract_color_identity() method
print("\n" + "="*70)
print("TESTING COLOR IDENTITY EXTRACTION")
print("="*70)

test_card_names = [
    'Meren of Clan Nel Toth',
    'Lightning Bolt',
    'Sol Ring',
    'Counterspell'
]

for card_name in test_card_names:
    # Try exact match first, then case-insensitive
    card_matches = scorer.all_cards[scorer.all_cards['name'] == card_name]
    if len(card_matches) == 0:
        card_matches = scorer.all_cards[
            scorer.all_cards['name'].str.lower() == card_name.lower()
        ]
    
    if len(card_matches) > 0:
        card = card_matches.iloc[0].to_dict()
        colors = scorer.extract_color_identity(card)
        print(f"{card_name:40s} → {colors if colors else {'colorless'}}")
    else:
        print(f"{card_name:40s} → NOT FOUND in {len(scorer.all_cards)} cards")

print("="*70)


TESTING COLOR IDENTITY EXTRACTION
Meren of Clan Nel Toth                   → {'B', 'G'}
Lightning Bolt                           → {'R'}
Sol Ring                                 → {'colorless'}
Counterspell                             → {'U'}


In [5]:
# DEBUG: Check what's actually in the lookup
print("\n" + "="*70)
print("DEBUG: LOOKUP CONTENTS")
print("="*70)

print(f"Total entries in color_identity_lookup: {len(scorer.color_identity_lookup)}")

# Search for cards with "lightning" or "sol" in name
search_terms = ["lightning", "sol ring"]
for term in search_terms:
    matching_keys = [k for k in scorer.color_identity_lookup.keys() if term.lower() in k.lower()]
    print(f"\nCards matching '{term}':")
    if matching_keys:
        for key in matching_keys[:5]:  # Show first 5
            print(f"  - '{key}': {scorer.color_identity_lookup[key]}")
    else:
        print(f"  (none found)")

# Check if they're in all_cards
print(f"\n'Lightning Bolt' in all_cards: {len(scorer.all_cards[scorer.all_cards['name'].str.contains('Lightning Bolt', case=False, na=False)]) > 0}")
print(f"'Sol Ring' in all_cards: {len(scorer.all_cards[scorer.all_cards['name'].str.contains('Sol Ring', case=False, na=False)]) > 0}")

print("="*70)


DEBUG: LOOKUP CONTENTS
Total entries in color_identity_lookup: 35510

Cards matching 'lightning':
  - 'Lightning Shrieker': {'R'}
  - 'Homing Lightning': {'R'}
  - 'Lightning Elemental': {'R'}
  - 'Living Lightning': {'R'}
  - 'Lightning Axe': {'R'}

Cards matching 'sol ring':
  - 'Sol Ring': set()
  - 'Sol Ring // Sol Ring': set()

'Lightning Bolt' in all_cards: True
'Sol Ring' in all_cards: True


In [6]:
# Test strict color identity enforcement
print("\n" + "="*70)
print("COLOR IDENTITY STRICT ENFORCEMENT TEST")
print("="*70)

# Test commander: Meren (Black/Green)
meren_matches = scorer.all_cards[scorer.all_cards['name'] == 'Meren of Clan Nel Toth']
if len(meren_matches) == 0:
    print("ERROR: Meren not found in database")
else:
    meren = meren_matches.iloc[0].to_dict()
    meren_colors = scorer.extract_color_identity(meren)
    
    print(f"\nCommander: Meren of Clan Nel Toth")
    print(f"Color identity: {meren_colors}")
    print(f"Legal colors: Black (B), Green (G), Colorless")
    print(f"Database has {len(scorer.all_cards)} cards total")
    
    # Test cards from database
    test_cards = [
        ('Lightning Bolt', False),        # Red - should be illegal
        ('Counterspell', False),          # Blue - should be illegal
        ('Sol Ring', True),               # Colorless - should be legal
        ('Swords to Plowshares', False),  # White - should be illegal
        ('Llanowar Elves', True),         # Green - should be legal
    ]
    
    print(f"\nTesting color identity enforcement:")
    all_passed = True
    found_count = 0
    
    for card_name, expected_legal in test_cards:
        card_matches = scorer.all_cards[scorer.all_cards['name'] == card_name]
        
        if len(card_matches) > 0:
            found_count += 1
            card = card_matches.iloc[0].to_dict()
            card_colors = scorer.extract_color_identity(card)
            multiplier = scorer._calculate_color_multiplier(card_colors, meren_colors)
            
            # Check if result matches expectation
            is_legal = multiplier > 0.0
            
            status = "✓ PASS" if (is_legal == expected_legal) else "✗ FAIL"
            if is_legal != expected_legal:
                all_passed = False
            
            color_str = str(card_colors) if card_colors else "colorless"
            expected_str = "LEGAL" if expected_legal else "ILLEGAL"
            print(f"  {status} {card_name:30s} colors={color_str:20s} multiplier={multiplier:.2f} (expected: {expected_str})")
        else:
            print(f"  ⚠ SKIP {card_name:30s} (not in database)")
    
    print("\n" + "="*70)
    print(f"Found {found_count}/{len(test_cards)} test cards")
    if all_passed and found_count > 0:
        print("✓✓✓ COLOR IDENTITY TEST PASSED ✓✓✓")
    elif found_count == 0:
        print("⚠ No test cards found in database")
    else:
        print("✗✗✗ COLOR IDENTITY TEST FAILED ✗✗✗")
    print("="*70)



COLOR IDENTITY STRICT ENFORCEMENT TEST

Commander: Meren of Clan Nel Toth
Color identity: {'B', 'G'}
Legal colors: Black (B), Green (G), Colorless
Database has 110031 cards total

Testing color identity enforcement:
  ✓ PASS Lightning Bolt                 colors={'R'}                multiplier=0.00 (expected: ILLEGAL)
  ✓ PASS Counterspell                   colors={'U'}                multiplier=0.00 (expected: ILLEGAL)
  ✓ PASS Sol Ring                       colors=colorless            multiplier=1.00 (expected: LEGAL)
  ✓ PASS Swords to Plowshares           colors={'W'}                multiplier=0.00 (expected: ILLEGAL)
  ✓ PASS Llanowar Elves                 colors={'G'}                multiplier=1.00 (expected: LEGAL)

Found 5/5 test cards
✓✓✓ COLOR IDENTITY TEST PASSED ✓✓✓


## Section 3: Debug Mechanic Synergy Calculation

Test mechanic extraction and identify why synergy always returns 100.

In [7]:
# Test mechanic extraction
print("\n" + "="*70)
print("MECHANIC EXTRACTION TEST")
print("="*70)

test_commanders = [
    'Meren of Clan Nel Toth',
    'Atraxa, Praetors\' Voice', 
    'The Gitrog Monster'
]

commander_mechanics = {}

for commander_name in test_commanders:
    card_matches = scorer.all_cards[
        scorer.all_cards['name'].str.lower().str.strip() == commander_name.lower().strip()
    ]
    if len(card_matches) > 0:
        card = card_matches.iloc[0].to_dict()
        mechanics_count = float(card.get('mechanic_count', 0))
        cluster = card.get('cluster_name', 'N/A')
        archetypes = card.get('archetypes_active', 'N/A')
        
        commander_mechanics[commander_name] = mechanics_count
        
        print(f"\n{commander_name}:")
        print(f"  Mechanic count: {mechanics_count}")
        print(f"  Cluster: {cluster}")
        print(f"  Archetypes: {archetypes}")
    else:
        print(f"\n{commander_name}: NOT FOUND")

print("\n" + "="*70)


MECHANIC EXTRACTION TEST

Meren of Clan Nel Toth:
  Mechanic count: 2.0
  Cluster: N/A
  Archetypes: N/A

Atraxa, Praetors' Voice:
  Mechanic count: 3.0
  Cluster: N/A
  Archetypes: N/A

The Gitrog Monster:
  Mechanic count: 2.0
  Cluster: N/A
  Archetypes: N/A



In [8]:
# Test synergy calculation with detailed logging
print("\n" + "="*70)
print("SYNERGY CALCULATION DEBUG")
print("="*70)

# Get Meren
meren_matches = scorer.all_cards[
    scorer.all_cards['name'].str.lower().str.strip() == 'meren of clan nel toth'.lower().strip()
]
if len(meren_matches) > 0:
    meren = meren_matches.iloc[0].to_dict()
    
    # Test synergy for a few cards
    test_card_names = [
        'Sol Ring',
        'Sakura-Tribe Elder',
        'Counterspell',
        'Crucible of Worlds'
    ]
    
    print(f"\nTesting synergy calculation for Meren:")
    
    synergy_scores = []
    for card_name in test_card_names:
        card_matches = scorer.all_cards[
            scorer.all_cards['name'].str.lower().str.strip() == card_name.lower().strip()
        ]
        if len(card_matches) > 0:
            card = card_matches.iloc[0].to_dict()
            synergy = scorer._calculate_mechanic_synergy(card, meren, [])
            synergy_scores.append(synergy)
            print(f"  {card_name:30s} synergy={synergy:6.2f}")
        else:
            print(f"  {card_name:30s} NOT FOUND")
    
    if synergy_scores:
        avg_synergy = np.mean(synergy_scores)
        min_synergy = np.min(synergy_scores)
        max_synergy = np.max(synergy_scores)
        print(f"\nSynergy statistics:")
        print(f"  Average: {avg_synergy:.2f}")
        print(f"  Min: {min_synergy:.2f}")
        print(f"  Max: {max_synergy:.2f}")
        print(f"\n⚠ Issue: All synergy scores the same? {len(set(synergy_scores)) == 1}")

print("\n" + "="*70)


SYNERGY CALCULATION DEBUG

Testing synergy calculation for Meren:


DEBUG: 
=== SYNERGY DEBUG: Sol Ring vs Meren of Clan Nel Toth ===
DEBUG: Card mechanics: set()
DEBUG: Commander mechanics: {'group', 'large'}
DEBUG: Mechanic overlap score: 0.0
DEBUG: Final synergy: 0.0


  Sol Ring                       synergy=  0.00


DEBUG: 
=== SYNERGY DEBUG: Sakura-Tribe Elder vs Meren of Clan Nel Toth ===
DEBUG: Card mechanics: {'flying', 'cast'}
DEBUG: Commander mechanics: {'group', 'large'}
DEBUG: Mechanic overlap score: 0.0
DEBUG: Final synergy: 0.0


  Sakura-Tribe Elder             synergy=  0.00


DEBUG: 
=== SYNERGY DEBUG: Counterspell vs Meren of Clan Nel Toth ===
DEBUG: Card mechanics: {'group', 'large'}
DEBUG: Commander mechanics: {'group', 'large'}
DEBUG: Mechanic overlap score: 100.0
DEBUG: Perfect mechanic match bonus applied: 100.0
DEBUG: Final synergy: 100.0


  Counterspell                   synergy=100.00


DEBUG: 
=== SYNERGY DEBUG: Crucible of Worlds vs Meren of Clan Nel Toth ===
DEBUG: Card mechanics: {'group', 'large'}
DEBUG: Commander mechanics: {'group', 'large'}
DEBUG: Mechanic overlap score: 100.0
DEBUG: Perfect mechanic match bonus applied: 100.0
DEBUG: Final synergy: 100.0


  Crucible of Worlds             synergy=100.00

Synergy statistics:
  Average: 50.00
  Min: 0.00
  Max: 100.00

⚠ Issue: All synergy scores the same? False



## Section 4: Deduplicate Cards in Results

Verify that score_all_cards() properly deduplicates and filters illegal cards.

In [ ]:
# Score all cards for Meren and check deduplication
print("\n" + "="*70)
print("DEDUPLICATION AND FILTERING TEST")
print("="*70)

meren_matches = scorer.all_cards[
    scorer.all_cards['name'].str.lower().str.strip() == 'meren of clan nel toth'.lower().strip()
]
if len(meren_matches) > 0:
    meren = meren_matches.iloc[0].to_dict()
    
    print(f"\nScoring all cards for Meren...")
    
    # Score all cards
    results_df = scorer.score_all_cards(meren, [])
    
    print(f"\nResults:")
    print(f"  Total unique cards: {len(results_df)}")
    print(f"  Duplicate card names: {results_df['card_name'].duplicated().sum()}")
    print(f"  Cards with color_multiplier = 0: {(results_df['color_multiplier'] == 0).sum()}")
    
    # Check for illegal cards in top 100
    top_100 = results_df.head(100)
    illegal_in_top = (top_100['color_multiplier'] == 0).sum()
    print(f"  Illegal cards in top 100: {illegal_in_top}")
    
    if illegal_in_top == 0:
        print("\n✓ PASS: No illegal cards in top 100 recommendations")
    else:
        print(f"\n✗ FAIL: Found {illegal_in_top} illegal cards in top 100!")
    
    print("\n" + "="*70)


DEDUPLICATION AND FILTERING TEST


INFO: Scoring all cards for Meren of Clan Nel Toth



Scoring all cards for Meren...


INFO: After deduplication: 35510 unique cards (from 110031 total)
DEBUG: 
=== SYNERGY DEBUG: Forest vs Meren of Clan Nel Toth ===
DEBUG: Card mechanics: set()
DEBUG: Commander mechanics: {'group', 'large'}
DEBUG: Mechanic overlap score: 0.0
DEBUG: Final synergy: 0.0
DEBUG: 
=== SYNERGY DEBUG: Fury Sliver vs Meren of Clan Nel Toth ===
DEBUG: Card mechanics: {'flying', 'cast'}
DEBUG: Commander mechanics: {'group', 'large'}
DEBUG: Mechanic overlap score: 0.0
DEBUG: Final synergy: 0.0
DEBUG: 
=== SYNERGY DEBUG: Kor Outfitter vs Meren of Clan Nel Toth ===
DEBUG: Card mechanics: {'group', 'large'}
DEBUG: Commander mechanics: {'group', 'large'}
DEBUG: Mechanic overlap score: 100.0
DEBUG: Perfect mechanic match bonus applied: 100.0
DEBUG: Final synergy: 100.0
DEBUG: 
=== SYNERGY DEBUG: Spirit vs Meren of Clan Nel Toth ===
DEBUG: Card mechanics: set()
DEBUG: Commander mechanics: {'group', 'large'}
DEBUG: Mechanic overlap score: 0.0
DEBUG: Final synergy: 0.0
DEBUG: 
=== SYNERGY DEBUG: Siren Look

In [ ]:
# Check if CardScorer supports parallel scoring
print("\n" + "="*70)
print("PERFORMANCE CHECK")
print("="*70)

import time

meren_matches = scorer.all_cards[
    scorer.all_cards['name'].str.lower().str.strip() == 'meren of clan nel toth'.lower().strip()
]

if len(meren_matches) > 0:
    meren = meren_matches.iloc[0].to_dict()
    
    print(f"\nDatabase size: {len(scorer.all_cards):,} cards")
    print(f"Testing scoring performance...")
    
    # Time the scoring
    start_time = time.time()
    results_df = scorer.score_all_cards(meren, [])
    elapsed = time.time() - start_time
    
    cards_per_sec = len(results_df) / elapsed
    
    print(f"\nResults:")
    print(f"  Cards scored: {len(results_df):,}")
    print(f"  Time elapsed: {elapsed:.2f} seconds")
    print(f"  Speed: {cards_per_sec:,.0f} cards/second")
    
    if elapsed > 10:
        print(f"\n⚠ Performance issue detected!")
        print(f"  Consider parallelization to speed this up")
    
print("="*70)

In [ ]:
# Show top 20 recommendations
if len(results_df) > 0:
    print(f"\nTop 20 recommendations for Meren:")
    print("\n")
    
    display_cols = ['card_name', 'total_score', 'mechanic_synergy', 'archetype_fit', 'combo_bonus', 'color_multiplier']
    top_20 = results_df[display_cols].head(20)
    
    # Format for nice display
    for idx, row in top_20.iterrows():
        print(f"{idx+1:3d}. {row['card_name']:35s} Score={row['total_score']:6.2f} Synergy={row['mechanic_synergy']:6.2f} Color={row['color_multiplier']:.2f}")

NameError: name 'results_df' is not defined

DEBUG: {"message": "Server heartbeat started", "topologyId": {"$oid": "697439e28a2ac949e563c20c"}, "driverConnectionId": 1, "serverConnectionId": 1376177638, "serverHost": "mtgecorec-westus2.mongo.cosmos.azure.com", "serverPort": 10255, "awaited": false}
DEBUG: {"message": "Server heartbeat succeeded", "topologyId": {"$oid": "697439e28a2ac949e563c20c"}, "driverConnectionId": 1, "serverConnectionId": 1376177638, "serverHost": "mtgecorec-westus2.mongo.cosmos.azure.com", "serverPort": 10255, "awaited": false, "durationMS": 11.133075997349806, "reply": "{\"ismaster\": true, \"maxBsonObjectSize\": 16777216, \"maxMessageSizeBytes\": 48000000, \"maxWriteBatchSize\": 1000, \"localTime\": 1769224776607, \"logicalSessionTimeoutMinutes\": 30, \"maxWireVersion\": 21, \"tags\": {\"region\": \"West US 2\"}, \"hosts\": [\"mtgecorec-westus2.mongo.cosmos.azure.com:10255\"], \"setName\": \"globaldb\", \"setVersion\": 1, \"primary\": \"mtgecorec-westus2.mongo.cosmos.azure.com:10255\", \"me\": \"mtgecorec

## Section 5: Validate All Critical Tests

Run comprehensive validation to ensure all fixes are working correctly.

In [ ]:
# Comprehensive validation
print("\n" + "="*70)
print("COMPREHENSIVE VALIDATION")
print("="*70)

validation_results = {}

# Test 1: Color Identity Enforcement
print("\n1. COLOR IDENTITY ENFORCEMENT")
meren_matches = scorer.all_cards[
    scorer.all_cards['name'].str.lower().str.strip() == 'meren of clan nel toth'.lower().strip()
]
if len(meren_matches) > 0:
    meren = meren_matches.iloc[0].to_dict()
    meren_colors = scorer.extract_color_identity(meren)
    
    # Test with Lightning Bolt (Red)
    bolt_matches = scorer.all_cards[
        scorer.all_cards['name'].str.lower().str.strip() == 'lightning bolt'.lower().strip()
    ]
    if len(bolt_matches) > 0:
        bolt = bolt_matches.iloc[0].to_dict()
        bolt_colors = scorer.extract_color_identity(bolt)
        multiplier = scorer._calculate_color_multiplier(bolt_colors, meren_colors)
        
        test_1_pass = multiplier == 0.0
        validation_results['color_identity'] = test_1_pass
        
        if test_1_pass:
            print("  ✓ PASS: Red card returns multiplier = 0.0 for Black/Green commander")
        else:
            print(f"  ✗ FAIL: Red card returns multiplier = {multiplier} (expected 0.0)")
    else:
        print("  ⚠ SKIP: Lightning Bolt not found in database")
        validation_results['color_identity'] = None
else:
    print("  ⚠ SKIP: Meren not found in database")
    validation_results['color_identity'] = None

In [ ]:
# Test 2: Synergy Variance
print("\n2. SYNERGY VARIANCE")

meren_matches = scorer.all_cards[
    scorer.all_cards['name'].str.lower().str.strip() == 'meren of clan nel toth'.lower().strip()
]
if len(meren_matches) > 0:
    meren = meren_matches.iloc[0].to_dict()
    
    # Sample 100 cards and check synergy variance
    sample_cards = scorer.all_cards.sample(min(100, len(scorer.all_cards)), random_state=42)
    synergy_scores = []
    
    for _, card_row in sample_cards.iterrows():
        card = card_row.to_dict()
        synergy = scorer._calculate_mechanic_synergy(card, meren, [])
        synergy_scores.append(synergy)
    
    unique_scores = len(set(synergy_scores))
    min_synergy = min(synergy_scores)
    max_synergy = max(synergy_scores)
    
    test_2_pass = (unique_scores > 1) and (min_synergy < max_synergy)
    validation_results['synergy_variance'] = test_2_pass
    
    if test_2_pass:
        print(f"  ✓ PASS: Synergy scores vary (min={min_synergy:.1f}, max={max_synergy:.1f}, unique={unique_scores})")
    else:
        print(f"  ✗ FAIL: Synergy scores don't vary (all same value = {synergy_scores[0]})")
else:
    print("  ⚠ SKIP: Meren not found")
    validation_results['synergy_variance'] = None

In [ ]:
# Test 3: Deduplication
print("\n3. DEDUPLICATION")

meren_matches = scorer.all_cards[
    scorer.all_cards['name'].str.lower().str.strip() == 'meren of clan nel toth'.lower().strip()
]
if len(meren_matches) > 0:
    meren = meren_matches.iloc[0].to_dict()
    results_df = scorer.score_all_cards(meren, [])
    
    has_duplicates = results_df['card_name'].duplicated().sum() > 0
    test_3_pass = not has_duplicates
    validation_results['deduplication'] = test_3_pass
    
    if test_3_pass:
        print(f"  ✓ PASS: No duplicate card names in results ({len(results_df)} unique cards)")
    else:
        print(f"  ✗ FAIL: Found {results_df['card_name'].duplicated().sum()} duplicate card names")
else:
    print("  ⚠ SKIP: Meren not found")
    validation_results['deduplication'] = None

In [ ]:
# Test 4: Commander-Specific Results
print("\n4. COMMANDER-SPECIFIC RESULTS")

# Score for multiple commanders
commander_names = [
    'Meren of Clan Nel Toth',
    'Atraxa, Praetors\' Voice',
    'The Gitrog Monster'
]

commander_top_20s = {}
for cmd_name in commander_names:
    cmd_matches = scorer.all_cards[
        scorer.all_cards['name'].str.lower().str.strip() == cmd_name.lower().strip()
    ]
    if len(cmd_matches) > 0:
        cmd = cmd_matches.iloc[0].to_dict()
        cmd_results = scorer.score_all_cards(cmd, [])
        commander_top_20s[cmd_name] = set(cmd_results['card_name'].head(20).values)

if len(commander_top_20s) >= 2:
    # Compare first two commanders
    names = list(commander_top_20s.keys())
    top_20_1 = commander_top_20s[names[0]]
    top_20_2 = commander_top_20s[names[1]]
    
    intersection = len(top_20_1 & top_20_2)
    unique_ratio = 1.0 - (intersection / 20.0)
    
    test_4_pass = unique_ratio >= 0.5  # At least 50% different
    validation_results['commander_specific'] = test_4_pass
    
    if test_4_pass:
        print(f"  ✓ PASS: {unique_ratio*100:.1f}% unique cards between {names[0]} and {names[1]} top 20")
    else:
        print(f"  ✗ FAIL: Only {unique_ratio*100:.1f}% unique (expected ≥50%)")
else:
    print("  ⚠ SKIP: Not enough commanders found")
    validation_results['commander_specific'] = None

In [ ]:
# Final validation summary
print("\n" + "="*70)
print("VALIDATION SUMMARY")
print("="*70)

test_names = [
    ('Color Identity Enforcement', 'color_identity'),
    ('Synergy Variance', 'synergy_variance'),
    ('Deduplication', 'deduplication'),
    ('Commander-Specific Results', 'commander_specific')
]

passed = 0
failed = 0
skipped = 0

print()
for test_name, key in test_names:
    result = validation_results.get(key)
    if result is True:
        print(f"  ✓ PASS: {test_name}")
        passed += 1
    elif result is False:
        print(f"  ✗ FAIL: {test_name}")
        failed += 1
    else:
        print(f"  ⚠ SKIP: {test_name}")
        skipped += 1

print("\n" + "="*70)
print(f"Results: {passed} PASSED, {failed} FAILED, {skipped} SKIPPED")
print("="*70)

if failed == 0 and passed > 0:
    print("\n✓✓✓ ALL VALIDATION TESTS PASSED ✓✓✓")
elif failed > 0:
    print("\n✗✗✗ SOME VALIDATION TESTS FAILED ✗✗✗")
else:
    print("\n⚠ INSUFFICIENT DATA FOR VALIDATION")